# Multi-class Classification on Anonymized 'Adult' Dataset

This notebook contains and analyses the multi-class classification of k-anonymous 'Adult' dataset from UCI Machine Learning repository. 

k-anonymity is a property of a dataset that the information for each entry can not be distinguished from at least *k-1* other entries in the dataset. Algorith used to obtain k-anonymity of the 'Adult' dataset is SaNGreeA - a version of a greedy clustering algorithm. For our latter experiments we will use 10 different k-values, however in this notebook we focus more on preprocessing methods in order to obtain similar results as in paper "DO NOT DISTURB? Classifier Behavior on Perturbed Datasets".

We use 4 classifiers:
<ol>
    <li>Gradient Boosting</li>
    <li>Random Forst</li>
    <li>Logistic Regression</li>
    <li>Linear SVC</li>
</ol>

In [1021]:
target = 'marital-status'
print(target)

marital-status


In [1022]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import pickle

from sklearn import metrics, preprocessing, model_selection
from sklearn.ensemble import GradientBoostingClassifier as GradientBoosting, RandomForestClassifier as RandomForest
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [1023]:
k = 100
mode = "global"
measure = "utility"
rank = "worst"

In [1024]:
def read_anon_data(filename):
    filepath = "../data/output/ARX/" + target + "/"
    filepath += filename
    dataset = pd.read_csv(filepath, sep=";", na_values="*", engine='python', index_col=False)
    return dataset

Let's load and have a look at our anonymous data.

In [1025]:
dataset = read_anon_data("anonymized_k_" + str(k) + "_" + mode + "_" + measure + "_rank_" + str(rank) + ".csv")
del dataset['NodeID']
dataset=dataset.dropna(axis=1,how='all')
dataset=dataset.dropna(axis=0,how='all')
dataset.head()

education-num marital-status relationship   race     sex  capital-gain  \
0       [9, 17[       Divorced    Unmarried  Black  Female           0.0   
1       [9, 17[       Divorced    Unmarried  Black  Female           0.0   
2       [9, 17[       Divorced    Unmarried  Black  Female           0.0   
3       [9, 17[       Divorced    Unmarried  Black  Female           0.0   
4       [9, 17[       Divorced    Unmarried  Black  Female           0.0   

   capital-loss hours-per-week native-country income  
0           0.0       [33, 41[  United-States  <=50K  
1           0.0       [33, 41[  United-States  <=50K  
2           0.0       [33, 41[  United-States  <=50K  
3           0.0       [33, 41[  United-States  <=50K  
4           0.0       [33, 41[  United-States  <=50K

In [1026]:
# Preprocessing
def number_encode_features(ds):
    result = ds.copy()
    encoders = {}
    for feature in result.columns:
        if result.dtypes[feature] == np.object:
            encoders[feature] = preprocessing.LabelEncoder()
            result[feature] = encoders[feature].fit_transform(result[feature].astype(str))
    return result, encoders

dataset_encoded, encoders = number_encode_features(dataset)

In [1027]:
y = dataset_encoded[target]
X = dataset_encoded.drop(target, axis=1)

In [1028]:
# Scoring
def f1_micro(clf, X, y):
    # cross validation scores on number encoded data
    scores = model_selection.cross_val_score(clf, X, y, cv=10, scoring='f1_micro')
    print("F1 score: %0.2f (+/- %0.2f)" 
          % (scores.mean(), scores.std() * 2))
    return scores.mean()

In [1029]:
scores = {}

## 1. Gradient Boosting

In [1030]:
# Gradient Boosting
clf = GradientBoosting(random_state=0)

In [1031]:
scores['Gradient Boosting'] = f1_micro(clf, X, y)

F1 score: 0.85 (+/- 0.22)


## 2. Random Forest

In [1032]:
# Random Forest
clf = RandomForest(random_state=0)

In [1033]:
scores['Random Forest'] = f1_micro(clf, X, y)

F1 score: 0.82 (+/- 0.25)


## 3. Logistic Regression

In [1034]:
# Logistic Regression
clf = LogisticRegression(random_state=0)

In [1035]:
scores['Logistic Regression'] = f1_micro(clf, X, y)

F1 score: 0.82 (+/- 0.18)


## 4. Linear SVC

In [1036]:
# Linear SVC - binary attributes needed
clf = LinearSVC(random_state=0)

In [1037]:
scores['Linear SVC'] = f1_micro(clf, X, y)

F1 score: 0.77 (+/- 0.29)


In [1038]:
scores

{'Gradient Boosting': 0.8494945368490525,
 'Random Forest': 0.8219083299525007,
 'Logistic Regression': 0.8173964538761493,
 'Linear SVC': 0.7657373777538333}

### Saving scores

In [1039]:
filename = "../data/output/ARX/" + target + "/classification-res/anonymized_k_" + str(k) + "_" + mode + "_" + measure + "_rank_" + str(rank)
outfile = open(filename,'wb')

In [1040]:
pickle.dump(scores, outfile)
outfile.close()